# ДОМАШНЕЕ ЗАДАНИЕ №1. Spark RDD API

**Дисциплина:** Методы обработки больших данных  
**Студент:** Попов Я.Ю. 
**Группа:** ИУ6-31М

### Задание 1:

1. Рассчитайте расстояние от заданной точки (lat=55.751244, lng=37.618423) до каждого заведения общепита из набора данных. Выведите первые 10.
2. Рассчитайте расстояние между всеми заведениями общепита из набора данных. Выведите первые 10.
3. Выведите топ-10 наиболее близких и наиболее отдаленных заведений.

## Импорт библиотек и инициализация SparkContext

In [3]:
from pyspark import SparkContext, SparkConf
import csv
from math import radians, sin, cos, sqrt, atan2

conf = SparkConf().setAppName("DistanceCalculator").setMaster("local[*]")
sc = SparkContext(conf=conf)

/home/ubuntu/ML/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


## Функция для вычисления расстояния по формуле гаверсинуса

In [5]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Радиус Земли в километрах
    
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    
    # Формула гаверсинуса
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Расстояние
    return R * c

### Загрузка и парсинг данных

In [6]:
data = sc.textFile("places.csv")

def parse_csv_line(line):
    reader = csv.reader([line])
    return next(reader)

parsed_data = data.map(parse_csv_line)

print("Пример записи:", parsed_data.first())

Пример записи: ['21830', 'Шоколадница', '637379839', 'да', 'Шоколадница', 'кафе', 'Центральный административный округ', 'Басманный район', 'город Москва, Бауманская улица, дом 33/2, строение 1', '(495) 916-57-62', '48', 'нет', '37.678681276899106', '55.77302899555391', '{type=Point, coordinates=[37.6786812768991, 55.7730289955539]}']


## Функция извлечения координат из последнего поля

In [7]:
def extract_coordinates(row):
    coords_str = row[-1]
    # Очистка строки от лишних символов
    coords_str = coords_str.replace("{type=Point, coordinates=[", "").replace("]}", "").replace("\"", "")
    # Извлечение долготы и широты
    coords = coords_str.split(",")
    if len(coords) >= 2:
        try:
            lng = float(coords[0].strip())
            lat = float(coords[1].strip())
            name = row[1]  # Название заведения
            return (name, lat, lng)
        except:
            return None
    else:
        return None

In [8]:
# Фильтруем некорректные данные
coords_rdd = parsed_data.map(extract_coordinates).filter(lambda x: x is not None)

print(f"Всего загружено заведений: {coords_rdd.count()}")

Всего загружено заведений: 769


## Пункт 1 - Расстояние от заданной точки

In [13]:
# Заданная точка
given_point = (55.751244, 37.618423)

# Вычисляем расстояния от заданной точки до каждого заведения
def distance_from_point(row):
    name, lat, lng = row
    distance = haversine_distance(given_point[0], given_point[1], lat, lng)
    return (name, lat, lng, distance)

# Вычисляем расстояния
distances_from_given_point = coords_rdd.map(distance_from_point)

# Выводим первые 10 результатов
print("Первые 10 расстояний от заданной точки:")
for result in distances_from_given_point.take(10):
    print(f"Заведение: {result[0]} \t Координаты: ({round(result[1], 6)}, {round(result[2], 6)}) \t Расстояние: {round(result[3], 2)} км")

print("\nТоп-10 ближайших заведений к точке (55.751244, 37.618423)")
top10_closest = distances_from_given_point.sortBy(lambda x: x[3]).take(10)
for i, (name, lat, lng, d) in enumerate(top10_closest, 1):
    print(f"{i:2}. {name:<25} | ({lat:.6f}, {lng:.6f}) | {d:.3f} км")    
    

Первые 10 расстояний от заданной точки:
Заведение: Шоколадница 	 Координаты: (55.773029, 37.678681) 	 Расстояние: 4.48 км
Заведение: МУ-МУ 	 Координаты: (55.771587, 37.67896) 	 Расстояние: 4.41 км
Заведение: КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА 	 Координаты: (55.765455, 37.679609) 	 Расстояние: 4.14 км
Заведение: Дом 12 	 Координаты: (55.762609, 37.661498) 	 Расстояние: 2.98 км
Заведение: Чито-Ра 	 Координаты: (55.762201, 37.662846) 	 Расстояние: 3.03 км
Заведение: Бар- буфет «Николай» 	 Координаты: (55.764501, 37.657009) 	 Расстояние: 2.83 км
Заведение: Флорентини 	 Координаты: (55.764827, 37.657776) 	 Расстояние: 2.89 км
Заведение: Beer Gik 	 Координаты: (55.762071, 37.634411) 	 Расстояние: 1.57 км
Заведение: Погребок 	 Координаты: (55.762071, 37.634411) 	 Расстояние: 1.57 км
Заведение: Пробка Гриль 	 Координаты: (55.762488, 37.635631) 	 Расстояние: 1.65 км

Топ-10 ближайших заведений к точке (55.751244, 37.618423)
 1. Мареа                     | (55.756835, 37.632064) | 1.056 км
 2.

## Пункт 2 - Расстояния между всеми заведениями

In [18]:
# Создаем broadcast-переменную со всеми координатами для оптимизации
coords_list = coords_rdd.collect()
coords_broadcast = sc.broadcast(coords_list)

# Вычисляем расстояния между всеми парами заведений
def all_distances(row):
    name1, lat1, lng1 = row
    results = []
    for name2, lat2, lng2 in coords_broadcast.value:
        if name1 != name2:  # Исключаем расстояние до самого себя
            distance = haversine_distance(lat1, lng1, lat2, lng2)
            results.append((name1, name2, distance))
    return results


all_cafe_distances = coords_rdd.flatMap(all_distances)

# Выводим первые n результатов
n = 30
print(f"\nПервые {n} расстояний между заведениями:")
for result in all_cafe_distances.take(n):
    print(f"{result[0]} --- {result[1]}: {round(result[2], 2)} км")


Первые 30 расстояний между заведениями:
Шоколадница --- МУ-МУ: 0.16 км
Шоколадница --- КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА: 0.84 км
Шоколадница --- Дом 12: 1.58 км
Шоколадница --- Чито-Ра: 1.56 км
Шоколадница --- Бар- буфет «Николай»: 1.65 км
Шоколадница --- Флорентини: 1.59 км
Шоколадница --- Beer Gik: 3.03 км
Шоколадница --- Погребок: 3.03 км
Шоколадница --- Пробка Гриль: 2.94 км
Шоколадница --- TEMPO DI PASTA: 2.94 км
Шоколадница --- Хлеб насущный: 2.94 км
Шоколадница --- Крошка Картошка: 2.62 км
Шоколадница --- Макдоналдс: 2.66 км
Шоколадница --- Теремок: 2.64 км
Шоколадница --- LAMBIC Брассерия: 2.49 км
Шоколадница --- ЭЛЬ ГАУЧО: 2.08 км
Шоколадница --- Папа Вейдер: 3.24 км
Шоколадница --- Темпл бар: 0.22 км
Шоколадница --- КАФЕ БУЛОШНАЯ: 2.24 км
Шоколадница --- Глав Пив Маг: 2.42 км
Шоколадница --- Beermood: 2.42 км
Шоколадница --- Чайный клуб «Мойчай.ру»: 2.06 км
Шоколадница --- Бейрут: 1.86 км
Шоколадница --- Cezve Coffee: 2.43 км
Шоколадница --- Плов: 2.81 км
Шоколадница ---

## Пункт 3 - Топ-10 близких и отдаленных заведений

In [19]:
# Топ-10 наиболее близких заведений
closest_pairs = all_cafe_distances.sortBy(lambda x: x[2]).take(10)
print("\nТоп-10 наиболее близких заведений:")
for pair in closest_pairs:
    print(f"{pair[0]} --- {pair[1]}: {round(pair[2], 2)} км")

# Топ-10 наиболее отдаленных заведений
farthest_pairs = all_cafe_distances.sortBy(lambda x: x[2], ascending=False).take(10)
print("\nТоп-10 наиболее отдаленных заведений:")
for pair in farthest_pairs:
    print(f"{pair[0]} --- {pair[1]}: {round(pair[2], 2)} км")



Топ-10 наиболее близких заведений:
Beer Gik --- Погребок: 0.0 км
Beer Gik --- Kozlovna: 0.0 км
Beer Gik --- Па-Паэлья: 0.0 км
Погребок --- Beer Gik: 0.0 км
Погребок --- Kozlovna: 0.0 км
Погребок --- Па-Паэлья: 0.0 км
TEMPO DI PASTA --- Хлеб насущный: 0.0 км
Хлеб насущный --- TEMPO DI PASTA: 0.0 км
Глав Пив Маг --- Beermood: 0.0 км
Beermood --- Глав Пив Маг: 0.0 км

Топ-10 наиболее отдаленных заведений:
МНИТИ --- Calabash Club: 5.46 км
МНИТИ --- Залечь на дно: 5.46 км
МНИТИ --- Политех: 5.46 км
МНИТИ --- Антикафе Checkpoint: 5.46 км
Calabash Club --- МНИТИ: 5.46 км
Залечь на дно --- МНИТИ: 5.46 км
Политех --- МНИТИ: 5.46 км
Антикафе Checkpoint --- МНИТИ: 5.46 км
МНИТИ --- Шоколадница: 5.4 км
Шоколадница --- МНИТИ: 5.4 км


## Останавливаем SparkContext

In [21]:
sc.stop()